In [ ]:
import numpy as np # linear algebra
import pandas as pd #
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

data = pd.read_excel("traffic detection.xlsx")
data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'traffic detection.xlsx'

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.isnull().any()

there are no null values


In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,10))
sns.swarmplot(y=data["total"], x = data["Traffic Situation"])

In [ ]:
plt.figure(figsize=(10,10))
sns.swarmplot(y=data["CarCount"], x = data["Traffic Situation"])

Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

data["Day"] = encoder.fit_transform(data["Day of the week"])
data["Traffic_Situation"] = encoder.fit_transform(data["Traffic Situation"])
data.head()

dropping the not required columns

In [ ]:
data.drop(["Day of the week","Traffic Situation"],inplace = True, axis = 1)

In [ ]:
data.head()

In [ ]:
data['Time'] = pd.to_datetime(data['Time'].astype(str))

# Extract the hour and minute components and convert 'Time' to minutes since midnight
data['Time'] = data['Time'].dt.hour * 60 + data['Time'].dt.minute

# Convert 'Time' to hours
data['Time'] = data['Time'] / 60.0

# Display the first few rows of the DataFrame
data.head()


outlier detection


In [ ]:
# Selecting the columns of interest
columns_of_interest = ['CarCount', 'BikeCount', 'TruckCount', 'suv count', 'total']

# Plot boxplots for outlier detection
plt.figure(figsize=(12, 8))
for i, column in enumerate(columns_of_interest, 1):
    plt.subplot(2, 3, i)
    data.boxplot(column=column)
    plt.title(f'Boxplot for {column}')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
Q1=np.percentile(data['BikeCount'],25, interpolation = 'midpoint')
Q2=np.percentile(data['BikeCount'],50, interpolation = 'midpoint')
Q3=np.percentile(data['BikeCount'],75, interpolation = 'midpoint')
Q2

In [ ]:
IQR=Q3-Q1
IQR

In [ ]:
lowlim=Q1-1.5*IQR
upplim=Q3+1.5*IQR
print(lowlim)
print(upplim)

In [ ]:
outliers = []
for i in data['BikeCount']:
    if i > upplim or i < lowlim:
        outliers.append(i)

print(outliers)


In [ ]:
import numpy as np
Q1=np.percentile(data['TruckCount'],25, interpolation = 'midpoint')
Q2=np.percentile(data['TruckCount'],50, interpolation = 'midpoint')
Q3=np.percentile(data['TruckCount'],75, interpolation = 'midpoint')
Q2

In [ ]:
IQR=Q3-Q1
IQR

In [ ]:
lowlim=Q1-1.5*IQR
upplim=Q3+1.5*IQR
print(lowlim)
print(upplim)

In [ ]:
import numpy as np

def handle_outliers_iqr(data, multiplier=1.5):
    # Calculate the first and third quartiles (Q1 and Q3)
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)

    # Calculate the IQR (Interquartile Range)
    IQR = Q3 - Q1

    # Define the lower and upper bounds to identify outliers
    lowlim = Q1 - multiplier * IQR
    upplim = Q3 + multiplier * IQR

    # Identify outliers
    outliers = (data < lowlim) | (data > upplim)

    # Handle outliers (you can choose a method like removing, replacing, or transforming)
    # In this example, we replace outliers with the median value
    data[outliers] = np.median(data)

    return data

# Example usage
data = np.array([54, 59, 57, 58, 56, 53, 55, 57, 54, 53, 53, 60, 55, 57, 57, 57, 57, 58, 57, 56, 53, 59, 58, 56])
print("Original data:", data)

# Handle outliers using IQR method
data_handled = handle_outliers_iqr(data)

print("Data after handling outliers:", data_handled)


#minmaxscaling


In [ ]:
print(type(data))


In [ ]:
import pandas as pd

# Convert the NumPy array to a Pandas DataFrame
data = pd.DataFrame(data)

# Now you can access columns
print(data.columns)


In [ ]:
print(data.columns)


In [ ]:
import pandas as pd

# Assuming data is a DataFrame or a NumPy array converted to a DataFrame
data = pd.DataFrame(data)

# Check the available column names
print(data.columns)

# If 'TruckCount' is not in the columns, investigate the correct column names
# Correct the column names if needed and then apply pd.to_numeric
data['BikeCount'] = pd.to_numeric(data['BikeCount'], errors='coerce')

# Check again after correcting column names
print(data.columns)

# Now apply pd.to_numeric for 'TruckCount'
data['TruckCount'] = pd.to_numeric(data['TruckCount'], errors='coerce')


In [ ]:
data['BikeCount'] = pd.to_numeric(data['BikeCount'], errors='coerce')
data['TruckCount'] = pd.to_numeric(data['TruckCount'], errors='coerce')


In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Initialize the MinMaxScaler
scaler = MinMaxScaler()
# Select columns to be scaled
columns_to_scale = ['BikeCount','TruckCount']
# Fit and transform the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Check the updated DataFrame
print(data.head())

#Random Forest Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Load the data
data = pd.read_excel("traffic detection.xlsx")

# Label Encoding
encoder = LabelEncoder()
data["Day"] = encoder.fit_transform(data["Day of the week"])
data["Traffic_Situation"] = encoder.fit_transform(data["Traffic Situation"])

# Drop unnecessary columns
data.drop(["Day of the week", "Traffic Situation"], inplace=True, axis=1)

# Convert 'Time' to hours
data['Time'] = pd.to_datetime(data['Time'].astype(str))
data['Hour'] = data['Time'].dt.hour
data['Minute'] = data['Time'].dt.minute
data['Time'] = data['Hour'] * 60 + data['Minute']
data.drop(['Hour', 'Minute'], inplace=True, axis=1)

# Min-max scaling
scaler = MinMaxScaler()
columns_to_scale = ['BikeCount', 'TruckCount']
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Define features (X) and target variable (y)
features = data.drop(['Traffic_Situation'], axis=1)
target = data['Traffic_Situation']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Drop datetime columns if any
X_train = X_train.select_dtypes(exclude=['datetime64'])
X_test = X_test.select_dtypes(exclude=['datetime64'])

# Initialize the Random Forest classifier
model1 = RandomForestClassifier(n_estimators=500, random_state=42)
model1.fit(X_train, y_train)
predictions = model1.predict(X_test)

# Evaluate the model
accuracy = (accuracy_score(y_test, predictions)*100)
conf_matrix = confusion_matrix(y_test, predictions)
class_report = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')